# Gas Dosing Demo

I created a makefile to make interacting with the Demo easier: 

Start Docker containers with the SEC Nodes running on port 10800 (gas_dosing) and 10801 (reactorcell):

 ``` make sim```

Start Containers and additionally start frappy gui:

```make frappy``` 

In [ ]:
import numpy as np
from bluesky import RunEngine
import bluesky.plan_stubs as bps
from bluesky.plan_stubs import sleep, rd
from bluesky.plans import scan, count


import databroker

from bluesky.preprocessors import run_decorator, SupplementalData
from bluesky import preprocessors as bpp
from bluesky.callbacks.best_effort import BestEffortCallback
from pprint import pprint
from secop_ophyd.SECoPDevices import SECoPNodeDevice, SECoPReadableDevice
from secop_ophyd.SECoPSignal import SECoPParamBackend
from bluesky.utils import ProgressBarManager

from tiled.client import from_uri
import NexusCreator

import time
from ophyd.status import Status


from bluesky.log import config_bluesky_logging

run = True

config_bluesky_logging(level='WARNING')
# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({},call_returns_result=True)
bec = BestEffortCallback()
RE.subscribe(bec)
RE.waiting_hook = ProgressBarManager()
RE.ignore_callback_exceptions = False



#Example of adding metadata to RE environment
investigation_id = "Nexus Demonstrator"

RE.md["investigation_id"] = investigation_id


client = from_uri("http://localhost:8000",api_key="secret")

def post_document(name,doc):
    client.post_document(name, doc)
    
RE.subscribe(post_document)

# Connect to Gas Dosing SEC Node and generate ophyd device tree
gas_dosing =  SECoPNodeDevice.create('localhost','10800',RE.loop)

# Connect to Reactor Cell SEC Node and generate ophyd device tree
reactor_cell =  SECoPNodeDevice.create('localhost','10801',RE.loop)

gas_dosing.class_from_instance()
reactor_cell.class_from_instance()


from genNodeClass import *

gas_dosing:Gas_dosing = gas_dosing
reactor_cell:Reactor_cell = reactor_cell

baseline = [gas_dosing.massflow_contr1.ramp]

sd = SupplementalData(baseline=baseline)

RE.preprocessors.append(sd)

In [ ]:
from typing_extensions import Self
from ophyd_async.core import Signal


class SECOP2NEXUS():
    @classmethod
    async def create(cls,nodes:list[SECoPNodeDevice]) -> Self:
        converter = SECOP2NEXUS(nodes)

        converter.group_names = [(await node.equipment_id.get_value()).split('.')[0] for node in nodes]

        for node in nodes:
            for name,module in node.mod_devices.items():
                if hasattr(module,'meaning'):
                    meaning_arr = await module.meaning.get_value()
                    meaning:dict  = meaning_arr.item() 

                    new_element:tuple = (meaning['importance'],module) 

                    if converter.meaning_dict.__contains__(meaning['function']):

                        converter.meaning_dict[meaning['function']].append(new_element)

                    else:
                        converter.meaning_dict[meaning['function']] = [new_element] 

        ## main part of nx_struct
        await converter.gen_general_data_struct()     

        ## sample partof nx_struct
        await converter.gen_sample_data_struct()

        return converter

        



    def __init__(self,nodes:list[SECoPNodeDevice]) -> None:
        self.nodes:list[SECoPNodeDevice] = nodes
        self.meaning_dict:dict[str,list[tuple[float,SECoPReadableDevice]]] = {}
        self.group_names:list[str] = []

        self.header:str = """
entry:NXentry
\t@NX_class = "NXentry"
"""

        self.general_data_struct:str= ""
        self.sample_data_struct:str = ""

    async def _generate_module_nexus_struct(self,module:SECoPReadableDevice) -> str:

        implementation: str = str(await module.implementation.get_value())
        description: str = str(await module.description.get_value())

        measurement_line = ""
        importance_line = ""
        key_line = ""
        link_line = ""

        # TODO handle legacy meaning correctly
        if hasattr(module,'meaning'):
            meaning_arr = await module.meaning.get_value()

            meaning: dict = meaning_arr.item()

            if meaning.get("function"):
                function: str = meaning.get("function", "")
                measurement_line = f'\n\tmeasurement:NX_CHAR = "{function}"'

                importance_line = (
                    f"\n\t\t@secop_meaning_importance = {meaning.get('importance')}"
                    if meaning.get("importance")
                    else ""
                )
                key_line = (
                    f"\n\t\t@long_name = \"{meaning.get('key')}\""
                    if meaning.get("key")
                    else ""
                )
                link_line = (
                    f"\n\t\t@PID = \"{meaning.get('link')}\""
                    if meaning.get("link")
                    else ""
                )

                # remove new line chars
        description = "".join(description.splitlines())

        text = f"""
{module._module}:NXsensor
\t@NX_class = NXsensor
\tname:NX_CHAR = "{module._module}"{measurement_line}{importance_line}{key_line}{link_line} 
\tmodel:NX_CHAR = "{implementation}"
\tdescription:NX_CHAR = "{description}"

"""
        value_log: str = await self._generate_nexus_parameter_log(module,"value")

        text += "\t".join(value_log.splitlines(True))

        text += "\tparameters:NXcollection"
        for parameter in module.param_devices.keys():
            if parameter == "value":
                continue

            param_log: str = await self._generate_nexus_parameter_log(module,parameter)

            text += "\t\t".join(param_log.splitlines(True))

        return text


    async def _generate_nexus_parameter_log(self,module:SECoPReadableDevice,param_name: str) -> str:

        sig: Signal = getattr(module, param_name)

        sig_backend: SECoPParamBackend = sig._backend

        # check if vlaue is numeric (NXlog only supports mumerical values)
        if not sig_backend.is_number():
            return ""

        param_unit = f'"{sig_backend.get_unit()}"'

        unit_line = (
            f"\n\t\t@units = {param_unit}" if sig_backend.get_unit() is not None else ""
        )

        log_name = "value_log" if param_name == "value" else param_name

        text = f"""
{log_name}:NXlog
\t@NX_class = NXlog
\tvalue:NX_FLOAT64 = {sig.name}{unit_line}
\t\t@type = "{sig_backend.describe_dict['SECoP_dtype']}"
\ttime:NX_FLOAT64 = {sig.name}-timestamp
\t\t@start = 0
\t\t@units = "s"

"""

        return text


    async def _generate_node_nexus_struct(self,node:SECoPNodeDevice) -> str:

        equipment_id: str = str(await node.equipment_id.get_value())

        if hasattr(node, "firmware"):
            firmware: str = str(await node.firmware.get_value())
        else:
            firmware = ""

        version: str = str(await node.version.get_value())
        description: str = str(await node.description.get_value())
        # remove newline chars
        description = "".join(description.splitlines())

        describe_message = str(node._secclient.client.descriptive_data).replace("\n", "")

        print(describe_message)

        text = f"""
{equipment_id}:NXenvironment
\t@NX_class = NXenvironment
\tname:NX_CHAR = "{equipment_id}"
\tshort_name:NX_CHAR = "{equipment_id.split('.')[0]}"
\ttype:NX_CHAR = "{firmware} ({version})"
\tdescription:NX_CHAR = "{description}"
\tsecop_describe_message:NX_CHAR = "{describe_message}"

"""
        for module in node.mod_devices.values():
            mod_str: str = await self._generate_module_nexus_struct(module) 

            text += "\t".join(mod_str.splitlines(True))

        return text


    async def gen_general_data_struct(self):
         ### Gerneral Data             
        self.general_data_struct = """
\tsample_environment:NXcollection
\t\t@NX_class = "NXcollection"
"""

        for node in self.nodes:
            node_text = await self._generate_node_nexus_struct(node=node)

            self.general_data_struct +=   "\t\t".join(node_text.splitlines(True))


    async def gen_sample_data_struct(self):
         ### Sample Data             
        self.sample_data_struct = """
\tsample:NXsample
\t\t@NX_class = "NXsample"
\t\ttype:NX_CHAR = "sample"
"""

        for group in self.group_names:
            self.sample_data_struct += f"""
\t\t{group}:NXenvironment
\t\t\t@NX_class = "NXenvironment"
\t\t\tname:NX_CHAR = "TODO"
"""

        for meaning , mod_list in self.meaning_dict.items():
            sorted_by_importance = sorted(mod_list, key=lambda tup: tup[0])
            
            closest_to_sample = sorted_by_importance[0]

            closest_module:SECoPReadableDevice = closest_to_sample[1]


            sig_backend: SECoPParamBackend = closest_module.value._backend

            # check if vlaue is numeric (NXlog only supports mumerical values)
            if not sig_backend.is_number():
                continue 

            param_unit = f'"{sig_backend.get_unit()}"'

            unit_line = (
                f"\n\t\t\t\t@units = {param_unit}" if sig_backend.get_unit() is not None else ""
            )
            #TODO do this with links
            self.sample_data_struct += f"""
\t\t{meaning}:NXlog
\t\t\t@NX_class = NXlog
\t\t\tvalue:NX_FLOAT64 = {closest_module.value.name}{unit_line}
\t\t\t\t@type = "{sig_backend.describe_dict['SECoP_dtype']}"
\t\t\ttime:NX_FLOAT64 = {closest_module.value.name}-timestamp
\t\t\t\t@start = 0
\t\t\t\t@units = "s"
"""     
        


    def write_nx_struct(self,path:str):
        with open(path, "w") as file:
            file.write(self.get_nexus_struct())

    def get_nexus_struct(self):

        return self.header + self.sample_data_struct + self.general_data_struct

conv = await SECOP2NEXUS.create(nodes= [gas_dosing,reactor_cell])


conv.write_nx_struct("demo.nxstruct")



In [ ]:

run = client[-1]

variables = {}

for key in run.primary.data.keys():
    if key == 'time':
        continue
    variables[key] = np.asarray(run.primary.data.get(key)).tolist()
    variables[key+'-timestamp'] = np.asarray(run.primary.data.get('time')).tolist()

config_dict:dict = run.primary.metadata['descriptors'][0]['configuration']

#pprint(config_dict)

for key,value in config_dict.items():
    data_dict:dict = value['data']
    postfix = '-meaning'

    data_dict = {key: value for key, value in data_dict.items() if not key.endswith(postfix)}

    variables.update(data_dict)

    timestamp_dict:dict = value['timestamps']
    variables.update ({k+'-timestamp': v for k, v in timestamp_dict.items()})


pprint(variables)

# Preparing NexusCreator
flags = {
  'nxstruct': './demo.nxstruct',
  'input': variables,
  'output': './minimal.nxs'
  
}


NexusCreator.executeConversion(flags)